In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import logging

from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("dair-ai/emotion", "split")
train = dataset.get("train")
val = dataset.get("validation")
test = dataset.get("test")
eval = concatenate_datasets([val, test])

In [ ]:
classes = len(train.unique("label"))
name = "google-bert/bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(name)
pad_id = tokenizer.get_vocab().get("[PAD]")

model = tnn.BertForClassification(classes=classes, hidden_size=1024, name=name)
lr = 1e-3
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id)

In [4]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=128,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process, batch_size=True, num_proc=2, remove_columns=["text"]
)
eval_dataset = eval.map(
    pre_process, batch_size=True, num_proc=2, remove_columns=["text"]
)

In [5]:
def collate_fn(batch):
    inputs = {"input_ids": [], "token_type_ids": [], "attention_mask": []}
    labels = []

    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["token_type_ids"].append(example["token_type_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["token_type_ids"] = torch.tensor(inputs["token_type_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [6]:
trainlaoder = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    eval_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=True)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [7]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainlaoder,
    testloader,
    epochs=10,
    unpack_inputs=True,
    save_weights=True,
    device=device,
    to_fn=to,
    path="../training/emotions-bert-sgdm.h5",
    verbose=True,
    profile=True,
)

In [ ]:
trainer.train()